<a href="https://colab.research.google.com/github/hoangphuc-03/baocao/blob/main/Doan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Các bước tiền xử lí

## 1.1 import các thư viện cần thiết

In [ ]:
%matplotlib inline
#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Data processing Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#huan luyen du lieu
from sklearn.preprocessing import LabelEncoder
#ma hoa gia tri sang so

from sklearn.preprocessing import StandardScaler
#chuan hoa du lieu

from sklearn.tree import DecisionTreeClassifier,plot_tree
#xay dung mo hinh cay
from sklearn.metrics import accuracy_score, confusion_matrix
#tinh toan do chinh xac mo hinh phan loai
from sklearn.metrics import accuracy_score

## 1.2 Cấu hình cài đặt

In [ ]:
pd.set_option('display.max_columns',100)          # Set now of columns visible to 100
pd.set_option('display.max_rows',100)

## 1.3 import file

In [ ]:
df=pd.read_csv("/content/Australian Vehicle Prices.csv")

# 2 | Dataframe Investigation

## 2.1 Basic Steps

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
# No Null values present
df.info()

## 2.2 Check Null and delet colum

In [ ]:
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
df = df.drop(columns=['BodyType','Doors','Seats'], axis=1)

In [ ]:
df.info()

In [ ]:
df=df.dropna()
null_counts = df.isnull().sum()
print(null_counts)

## 2.3 data transfer

In [ ]:
pd.value_counts(df.dtypes)

In [ ]:
#Tạo một đối tượng LabelEncoder
label_encoder = LabelEncoder()
df.loc[:, 'Brand'] = label_encoder.fit_transform(df['Brand'])
df.loc[:, 'Year'] = label_encoder.fit_transform(df['Year'])
df.loc[:, 'Model'] = label_encoder.fit_transform(df['Model'])
df.loc[:, 'Car/Suv'] = label_encoder.fit_transform(df['Car/Suv'])
df.loc[:, 'Title'] = label_encoder.fit_transform(df['Title'])
df.loc[:, 'UsedOrNew'] = label_encoder.fit_transform(df['UsedOrNew'])
df.loc[:, 'Transmission'] = label_encoder.fit_transform(df['Transmission'])
df.loc[:, 'Engine'] = label_encoder.fit_transform(df['Engine'])
df.loc[:, 'DriveType'] = label_encoder.fit_transform(df['DriveType'])
df.loc[:, 'FuelType'] = label_encoder.fit_transform(df['FuelType'])
df.loc[:, 'FuelConsumption'] = label_encoder.fit_transform(df['FuelConsumption'])
df.loc[:, 'Kilometres'] = label_encoder.fit_transform(df['Kilometres'])
df.loc[:, 'ColourExtInt'] = label_encoder.fit_transform(df['ColourExtInt'])
df.loc[:, 'Location'] = label_encoder.fit_transform(df['Location'])
df['CylindersinEngine'] = label_encoder.fit_transform(df['CylindersinEngine'])
df['Price'] = label_encoder.fit_transform(df['Price'])

#Hiển thị DataFrame sau khi chuyển đổi
df.describe()

In [ ]:
pd.value_counts(df.dtypes)

## 2.4 ma trận tương quang

In [ ]:
plt.figure(figsize=(20,15))
plt.title('Heatmap corralation')
sns.heatmap(data=pd.get_dummies(df).corr(), annot=True);

# 3 gán nhãn cho Price

In [ ]:
df['price_label'] = df['Price'].apply(lambda value: 'Dỏm' if value <= 742 else 'Trung bình' if value <= 1364 else 'Khá' if value <= 2242.25 else 'Tốt')

In [ ]:
df.info()

# 4 số biểu đồ

## 4.1 biểu đồ đếm

In [ ]:
plt.figure(figsize=(15,8))
plt.title('Count of Price')
sns.countplot(data=df, x='price_label')

## 4.2 Biểu đồ histogram

In [ ]:
df.hist(figsize=(15,15),bins=50);

# 5 modeling

In [ ]:
target = df.pop('Price')

In [ ]:
# Xác định các đặc trưng (X) và biến mục tiêu (y)
X = df.drop('price_label', axis=1)
y = df['price_label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5.1 Decision Tree

In [ ]:
# Initialize the Decision Tree classifier
#dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier = DecisionTreeClassifier(random_state=42)
# Train the model on the training set
dt_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dt_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy}")

In [ ]:
# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display the confusion matrix using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['price_label 0', 'price_label 1', 'price_label 2', 'price_label 3'], yticklabels=['price_label 0', 'price_label 1', 'price_label 2', 'price_label 3'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plot_tree(dt_classifier, filled=True, feature_names=X.columns, class_names=["price_label 0", "price_label 1", "price_label 2", "price_label 3"], max_depth=2)
plt.show()

In [ ]:
#Find the best max-depth for DT algorithm
# Define a range of max_depth values to iterate over
max_depth_values = range(3, 20)  # You can adjust this range

# Initialize variables to keep track of the best max depth and corresponding score
best_max_depth = None
best_score = 0.0

# Iterate over different max_depth values
for max_depth in max_depth_values:
    # Initialize the DecisionTreeClassifier with the current max_depth
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    # Fit the model to the training data
    clf.fit(X_train, y_train)
    # Evaluate the model on the test data
    test_score = clf.score(X_test, y_test)
    # Perform cross-validation on the training data
    #cv_score = cross_val_score(clf, X_train, y_train, cv=5).mean()

    # Print the score for the current max_depth
    print(f"Max Depth = {max_depth}: Test Score = {test_score}")
    # Check if the current model has a better score than the previous best
    if test_score > best_score:
        best_score = test_score
        best_max_depth = max_depth

# Print the best max depth and corresponding score
print(f"Best Max Depth = {best_max_depth}: Best Test Score = {best_score}")

In [ ]:
from sklearn.model_selection import cross_val_score
# Perform 10-fold cross-validation
scores = cross_val_score(clf, X, y, cv=10)
# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Mean CV Score:", scores.mean())

In [ ]:
from sklearn.tree import export_text
# Export the text representation of the decision tree
tree_rules = export_text(clf, feature_names=X.columns.tolist())
print(tree_rules)

## 5.2 Random Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#tao du lieu de thu
X = pd.get_dummies(X)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện mô hình trên tập dữ liệu huấn luyện
rf.fit(X_train, y_train)

# Dự đoán nhãn cho tập dữ liệu test
predictions = rf.predict(X_test)

# Đánh giá độ chính xác và hiển thị báo cáo
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import graphviz

# Khởi tạo và huấn luyện mô hình Random Forest
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)

# Xuất ra biểu đồ của ba cây quyết định đầu tiên
for i in range(3):
    tree = random_forest.estimators_[i]
    dot_data = export_graphviz(tree, feature_names=X_train.columns, filled=True, max_depth=2,impurity=False, proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)